# Check Raw Eye Tracking Data for POST-Decision Period

This notebook loads raw eye tracking data following the same approach as the preprocessing pipeline to check whether gaze data exists after the submit button press (POST-decision period).

In [48]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import re
from pathlib import Path

## Helper Functions (from preprocessing notebook)

In [49]:
def extract_subject_id(filename):
    """Extract subject ID from filename.
    
    Handles two formats:
    1. JSON: MMDD_HHMM_LCT_DESKTOP-XXXXXXX.json -> MMDD_HHMM_XXXXXXX
    2. PKL: MMDD_HHMM_XXXXXXX.pkl -> MMDD_HHMM_XXXXXXX
    """
    # Try format with DESKTOP (JSON files)
    match = re.search(r"(\d{4}_\d{4})_.*?DESKTOP-([A-Z0-9]+)", filename)
    if match:
        return f"{match.group(1)}_{match.group(2)}"
    
    # Try simple format (PKL files): MMDD_HHMM_XXXXXXX
    match = re.search(r"(\d{4}_\d{4}_[A-Z0-9]+)", filename)
    if match:
        return match.group(1)
    
    return None

def load_subject_data(json_path, pkl_path):
    """Load JSON and PKL files for a subject."""
    # Load JSON
    with open(json_path, 'r') as f:
        data = json.load(f)

    # Load PKL
    with open(pkl_path, 'rb') as f:
        eye_pkl = pickle.load(f)

    # Convert to DataFrame if needed
    if not isinstance(eye_pkl, pd.DataFrame):
        eye_pkl = pd.DataFrame(eye_pkl)

    return data, eye_pkl

## Load Data for One Session

In [50]:
# Pick a trial to examine
trial_idx = 0

if matching_json:
    trial = data['trials'][trial_idx]
    
    print(f"=" * 80)
    print(f"TRIAL {trial_idx}")
    print(f"=" * 80)
    
    # Extract submit time from LCT events
    lct = trial.get('lct', [])
    show_screen_time = None
    submit_time = None
    
    for event in lct:
        if 'show screen' in event['event']:
            show_screen_time = event['time']
        elif 'submit' in event['event']:
            submit_time = event['time']
    
    print(f"\nTimestamps from LCT events (ms):") 
    print(f"  Show screen: {show_screen_time}")
    print(f"  Submit time: {submit_time}")
    
    # Check embedded trial['eye'] data
    has_embedded_eye = 'eye' in trial and len(trial.get('eye', [])) > 0
    print(f"\n--- EMBEDDED trial['eye'] data ---")
    print(f"Has embedded eye data: {has_embedded_eye}")
    
    if has_embedded_eye and submit_time:
        eye_times = [sample['time'] for sample in trial['eye']]
        post_submit_samples = [s for s in trial['eye'] if s['time'] > submit_time]
        print(f"  Time range: {min(eye_times):.1f} to {max(eye_times):.1f} ms")
        print(f"  Submit: {submit_time:.1f} ms")
        print(f"  POST-decision samples: {len(post_submit_samples)}")
    
    # NOW CHECK THE RAW PKL FILE (like preprocessing does)
    print(f"\n--- RAW PKL FILE data (like preprocessing pipeline) ---")
    
    if has_embedded_eye and submit_time:
        # Match using eye_tracker_timestamp (following preprocessing logic)
        first_eye_timestamp = trial['eye'][0].get('eye_tracker_timestamp')
        
        if first_eye_timestamp is not None:
            # Find trial start in PKL
            matches = eye_pkl[np.isclose(eye_pkl['eye_tracker_timestamp'], 
                                         first_eye_timestamp, atol=1.0)]
            
            if len(matches) > 0:
                trial_start_time_pkl = matches.iloc[0]['time']
                print(f"  Matched trial start in PKL at time: {trial_start_time_pkl:.1f} ms")
                
                # Find submit in PKL (find closest sample to submit_time)
                closest_submit = min(trial['eye'], key=lambda x: abs(x['time'] - submit_time))
                submit_eye_timestamp = closest_submit.get('eye_tracker_timestamp')
                
                if submit_eye_timestamp is not None:
                    submit_matches = eye_pkl[np.isclose(eye_pkl['eye_tracker_timestamp'], 
                                                        submit_eye_timestamp, atol=1.0)]
                    
                    if len(submit_matches) > 0:
                        submit_time_pkl = submit_matches.iloc[0]['time']
                        print(f"  Submit time in PKL: {submit_time_pkl:.1f} ms")
                        
                        # Extract POST-decision window (following preprocessing: +2 seconds after submit)
                        trial_end_time = submit_time_pkl + 2000  # 2 seconds in ms
                        
                        post_decision_data = eye_pkl[
                            (eye_pkl['time'] > submit_time_pkl) & 
                            (eye_pkl['time'] <= trial_end_time)
                        ]
                        
                        print(f"  POST-decision window: {submit_time_pkl:.1f} to {trial_end_time:.1f} ms (2 sec)")
                        print(f"  POST-decision samples in PKL: {len(post_decision_data)}")
                        
                        if len(post_decision_data) > 0:
                            print(f"\n  ✓ POST-decision data EXISTS in raw PKL!")
                            print(f"  Duration: {(post_decision_data['time'].max() - submit_time_pkl)/1000:.3f} seconds")
                            
                            # Check pupil vs gaze
                            valid_pupil = ((post_decision_data['pupilDiameterL'] > 0) | 
                                          (post_decision_data['pupilDiameterR'] > 0)).sum()
                            non_zero_gaze = ((post_decision_data['gazeL_X'] != 0) | 
                                           (post_decision_data['gazeR_X'] != 0)).sum()
                            
                            print(f"    Pupil samples: {valid_pupil}/{len(post_decision_data)}")
                            print(f"    Gaze samples: {non_zero_gaze}/{len(post_decision_data)}")
                            
                            # Show first sample
                            print(f"\n  First POST-decision sample from PKL:")
                            first = post_decision_data.iloc[0]
                            print(f"    time: {first['time']:.1f}")
                            print(f"    pupilDiameterL: {first['pupilDiameterL']:.3f}")
                            print(f"    gazeL_X: {first['gazeL_X']:.6f}")
                            print(f"    gazeL_Y: {first['gazeL_Y']:.6f}")
                        else:
                            print(f"  ✗ NO POST-decision data in PKL")

TRIAL 0

Timestamps from LCT events (ms):
  Show screen: 607246.478656
  Submit time: 607249.1445494

--- EMBEDDED trial['eye'] data ---
Has embedded eye data: True
  Time range: 607246.5 to 607249.1 ms
  Submit: 607249.1 ms
  POST-decision samples: 0

--- RAW PKL FILE data (like preprocessing pipeline) ---
  Matched trial start in PKL at time: 99953.3 ms
  Submit time in PKL: 99956.0 ms
  POST-decision window: 99956.0 to 101956.0 ms (2 sec)
  POST-decision samples in PKL: 80667

  ✓ POST-decision data EXISTS in raw PKL!
  Duration: 0.672 seconds
    Pupil samples: 75329/80667
    Gaze samples: 75329/80667

  First POST-decision sample from PKL:
    time: 99956.0
    pupilDiameterL: 2.491
    gazeL_X: 0.083603
    gazeL_Y: -0.113052
